## Resampling Sentinel-2 Bands with Different Resolutions Using OpenEO
### Introduction

In geospatial analysis, working with multi-band data from satellite imagery often involves dealing with bands of varying resolutions. This is particularly true for Sentinel-2 data, where different bands have different spatial resolutions. To combine these bands into a single data cube for analysis, we must first resample them to a common resolution. In this tutorial, we will demonstrate how to load two Sentinel-2 bands with different resolutions, resample one of the bands, and create a data cube using the OpenEO Python client.
## Objective

### The objective of this tutorial is to:
- Load two Sentinel-2 bands with different spatial resolutions.
- Resample one of the bands to match the resolution of the other.
- Create a data cube that combines the two resampled bands.

### In this notebook, you will explore the following concepts:

A. **Loading Sentinel-2 Bands:** Using OpenEO to load Sentinel-2 bands with different resolutions.
B. **Resampling Data Cubes:** Applying resampling techniques to adjust the resolution of one band to match the other.
C. **Creating a Multi-band Data Cube**: Combining the resampled bands into a single data cube.

### Tools You'll Use:

- **OpenEO**: To interact with satellite data and perform spatial operations.
- **Xarray**: For managing and analyzing multi-dimensional data arrays.
- **Matplotlib**: For visualizing the results of your analysis.

### Learning Outcomes:

By the end of this tutorial, you will be able to:

- Load Sentinel-2 bands with different resolutions using OpenEO.
- Resample one band to match the resolution of another using various interpolation methods.
- Create and visualize a multi-band data cube.

In [1]:
from config import eo_service_url,user,passwd
from demo_regions import  s2

## A. Loading Sentinel-2 Bands

We start by connecting to the OpenEO backend and loading two Sentinel-2 bands: one with 10m resolution (e.g., B04) and another with 20m resolution (e.g., B11).

In [8]:
import openeo
import xarray as xr
import matplotlib.pyplot as plt

# Connect to OpenEO
#conn = openeo.connect(eo_service_url)
conn = openeo.connect("http://host.docker.internal:8000")

# Authenticate (replace with your credentials or use OAuth if available)
conn.authenticate_basic(username=user, password=passwd)

# Define the collection and bands to load
collection = s2.s2_msi_l2a
band_10m = "b04"  # Red band, 10m resolution
band_20m = "b11"  # SWIR band, 20m resolution

# Define a bounding box for the area of interest
bbox = {
          "west": 13.460930954751282,
          "east": 13.585449709728213,
          "south": 59.36327761288911,
          "north": 59.40107161575003
        }
timespan=["2024-06-12", "2024-06-13"]
# Load the data for both bands
datacube_10m = conn.load_collection(collection, bands=[band_10m], temporal_extent=timespan, spatial_extent=bbox)
datacube_20m = conn.load_collection(collection, bands=[band_20m], temporal_extent=timespan, spatial_extent=bbox)


Note that the above code only creates the process trees to send to the back-end. Nothing has been fetched yet.
## B. Resampling the 20m Band to 10m Resolution

Since the datacubes must have the same resolution, we need to resample the 20m band to match the 10m resolution of the other band


# TODO , swap so we sample 10m-->20m

In [9]:
# Resample the 20m datacube to 10m resolution
resampled_20m = datacube_20m.resample_cube_spatial(target=datacube_10m, method="bilinear")


### Interpolation Methods:

- **Nearest Neighbor** (`near`): Fastest but may result in blocky artifacts.
- **Bilinear** (`bilinear`): A good balance between performance and smoothing.
- **Cubic** (`cubic`): Produces smoother results but is computationally more expensive.
- **Cubic Spline** (`cubicspline`): Very smooth, suitable for continuous data; more computationally intensive.
- **Lanczos** (`lanczos`): Sharp, high-quality interpolation; more computationally expensive, good for resampling imagery.
- **Average** (`average`): Useful for downsampling; averages the values in the neighborhood.
- **Mode** (`mode`): Picks the most frequent value in the neighborhood; useful for categorical data.
- **Maximum** (`max`): Selects the maximum value in the neighborhood.
- **Minimum** (`min`): Selects the minimum value in the neighborhood.
- **Median** (`med`): Selects the median value in the neighborhood.
- **First Quartile** (`q1`): Selects the first quartile value in the neighborhood.
- **Third Quartile** (`q3`): Selects the third quartile value in the neighborhood.


You can choose any of these methods depending on your needs.

## C. Creating a Multi-band Data Cube

Now that both bands have the same resolution, we can combine them into a single data cube.

In [10]:
# Merge the datacubes into a single multi-band cube
multi_band_cube = datacube_10m.merge_cubes(resampled_20m)

multi_band_cube


In [11]:
# Optionally, visualize the combined data cube (e.g., for a single timestamp)
job = multi_band_cube.create_job(out_format='netcdf',title="Combine bands of different resolution")

In [12]:
display(job)

<BatchJob job_id='d3ddf9a8-8e17-43c4-b6aa-e9c1789904b5'>

In [13]:
job.start_and_wait()

0:00:00 Job 'd3ddf9a8-8e17-43c4-b6aa-e9c1789904b5': send 'start'
0:00:00 Job 'd3ddf9a8-8e17-43c4-b6aa-e9c1789904b5': queued (progress 0.0%)
0:00:05 Job 'd3ddf9a8-8e17-43c4-b6aa-e9c1789904b5': running (progress 0.0%)
0:00:11 Job 'd3ddf9a8-8e17-43c4-b6aa-e9c1789904b5': running (progress 0.0%)
0:00:19 Job 'd3ddf9a8-8e17-43c4-b6aa-e9c1789904b5': finished (progress 0.0%)


<BatchJob job_id='d3ddf9a8-8e17-43c4-b6aa-e9c1789904b5'>

In [16]:
job.logs()

[{'id': '889',
  'level': 'info',
  'message': 'Worker EOW-2024-09-12-14:39--dfm-nbs-YNPRYO Completed Job with State.finished.',
  'time': '2024-10-23T12:58:08.607793Z',
  'usage': {'duration': {'value': 12783.0, 'unit': 'ms'},
   'storage': {'value': 5059976.0, 'unit': 'bytes'}}},
 {'id': '886',
  'level': 'info',
  'message': 'Job Created',
  'time': '2024-10-23T12:57:51.366159Z'},
 {'id': '887',
  'level': 'info',
  'message': 'Job start execution requested.',
  'time': '2024-10-23T12:57:52.910489Z'},
 {'id': '888',
  'level': 'info',
  'message': 'Worker EOW-2024-09-12-14:39--dfm-nbs-YNPRYO Received Job for execution.',
  'time': '2024-10-23T12:57:55.681400Z'}]

In [15]:
job.get_results()

<JobResults for job 'd3ddf9a8-8e17-43c4-b6aa-e9c1789904b5'>

In [20]:
results = job.get_results()

In [24]:
results.download_files()

[PosixPath('/proj/tutorials/workshop_W2443/process.json'),
 PosixPath('/proj/tutorials/workshop_W2443/summary.nc'),
 PosixPath('/proj/tutorials/workshop_W2443/stac.json'),
 PosixPath('/proj/tutorials/workshop_W2443/py_proc_std_err.txt'),
 PosixPath('/proj/tutorials/workshop_W2443/py_proc_std_out.txt'),
 PosixPath('/proj/tutorials/workshop_W2443/d3ddf9a8-8e17-43c4-b6aa-e9c1789904b5.txt'),
 PosixPath('/proj/tutorials/workshop_W2443/job-results.json')]

In [26]:
xr.open_dataset("summary.nc")

<xarray.Dataset> Size: 5MB
Dimensions:      (t: 1, y: 438, x: 718)
Coordinates:
  * t            (t) datetime64[ns] 8B 2024-06-12T10:30:31.024000
  * y            (y) float64 4kB 6.586e+06 6.586e+06 ... 6.581e+06 6.581e+06
  * x            (x) float64 6kB 4.125e+05 4.125e+05 ... 4.197e+05 4.197e+05
    spatial_ref  int32 4B ...
Data variables:
    b04          (t, y, x) float64 3MB ...
    b11          (t, y, x) float64 3MB ...
Attributes:
    crs:         EPSG:3006
    nodata:      -9999
    collection:  s2_msi_l2a

## Conclusion

In this notebook, we've demonstrated how to load Sentinel-2 bands with different resolutions, resample one band to match the resolution of the other, and combine them into a single data cube using the OpenEO Python client. The choice of interpolation method for resampling can affect the results, so it's important to consider the trade-offs between speed and accuracy.
Next Steps

You can extend this workflow by applying further analyses to the multi-band data cube, such as computing vegetation indices or classifying land cover types.